In [ ]:
# PREFILLED
%config InlineBackend.figure_formats = ['svg']
import holoviews as hv
hv.extension('bokeh')
hv.output(widget_location='bottom')

# Imports

In [ ]:
import discretisedfield as df
import micromagneticmodel as mm
import numpy as np
import oommfc as mc

## System

In [ ]:
system = mm.System(name='dw_pair_conversion')

## Energy equation

In [ ]:
# PREFILLED
A_ex = 15e-12  # A/m

In [ ]:
system.energy = (mm.Exchange(A=A_ex)
                 + mm.DMI(D=3e-3, crystalclass='Cnv_z')
                 + mm.UniaxialAnisotropy(K=0.6e6, u=(0, 0, 1))
                 + mm.Demag()
                 + mm.Zeeman(H=(0, 0, 100e-3 / mm.consts.mu0))
                )

In [ ]:
system.energy

## Initial magnetisation

In [ ]:
region = df.Region(p1=(-100e-9, -50e-9, -10e-9), p2=(200e-9, 50e-9, 0))
mesh = df.Mesh(region=region, cell=(2.5e-9, 2.5e-9, 5e-9), subregions={'sr': region})

In [ ]:
4 * np.pi * A_ex / 3e-3

In [ ]:
# PREFILLED
Ms = 5.8e5  # A/m

In [ ]:
def Ms_func(p):
    x, y, z = p
    if x > 0 or abs(y) < 20e-9:
        return Ms
    return 0

def init_m(p):
    x, y, z = p
    if -60e-9 < x < -20e-9:
        return (0, 0, -1)
    return (0, 0, 1)

system.m = df.Field(mesh, dim=3, value=init_m, norm=Ms_func)

In [ ]:
system.m.plane('z').z.mpl()

## Energy minimisation

In [ ]:
md = mc.MinDriver()
md.drive(system)

In [ ]:
system.m.plane('z').z.mpl()

In [ ]:
system.m.hv(kdims=['x', 'y'], vector_kw={'n': (45, 45)})

## Applying in-plane current

In [ ]:
def J2u(J):
    muB = 9.27e-24
    return J * muB / (mm.consts.e * Ms)

In [ ]:
J2u(.6e12)

In [ ]:
J2u(5e12)

In [ ]:
system.dynamics = mm.Precession(gamma0=mm.consts.gamma0) + mm.Damping(alpha=0.3) + mm.ZhangLi(u=300, beta=0.5)
system.dynamics

In [ ]:
td = mc.TimeDriver()
td.drive(system, t=0.35e-9, n=200, verbose=2)

In [ ]:
system.m.hv(kdims=['x', 'y'], vector_kw={'n': (45, 15)}, scalar_kw={'clim': (-Ms, Ms), 'cmap': 'coolwarm'})

## Data analysis

In [ ]:
import micromagneticdata as mdata
data = mdata.Data(system.name)

In [ ]:
data.info

In [ ]:
data[-2].hv(kdims=['x', 'y'],
            vdims=['x', 'y'],
            vector_kw={'n' : (45, 15)},
            scalar_kw = {'clim':(-Ms, Ms), 'cmap': 'coolwarm'})

In [ ]:
system.dynamics -= system.dynamics.zhangli
td.drive(system, t=.5e-9, n=1, verbose=2)

In [ ]:
system.m.hv(kdims=['x', 'y'], vector_kw={'n': (45, 15)}, scalar_kw={'clim': (-Ms, Ms), 'cmap': 'coolwarm'})

## Simulate experiments with `mag2exp`

![Ubermag packages](overview.pmg)

In [ ]:
md.drive(system)

In [ ]:
system.m.hv(kdims=['x', 'y'], vector_kw={'n': (45, 15)}, scalar_kw={'clim': (-Ms, Ms), 'cmap': 'coolwarm'})

In [ ]:
import mag2exp

In [ ]:
mfm_m_field = system.m.pad({'z': (0, 25)}, mode='constant')

In [ ]:
system.m.mesh.region

In [ ]:
# PREFILLED
mfm_m_field.mesh.subregions = {
    #'sample': df.Region(p1=(-100e-9, -50e-9, -2.5e-9), p2=(200e-9, 50e-9, 0)),
    'free': df.Region(p1=(-100e-9, -50e-9, 5e-9), p2=(200e-9, 50e-9, 100e-9))
}

In [ ]:
mfm_m_field.plane('y').z.mpl()

In [ ]:
phase_shift = mag2exp.mfm.phase_shift(mfm_m_field, quality=650, k=3, tip_m=(0, 0, 1e-16))

In [ ]:
phase_shift['free'].hv(kdims=['x', 'y'], scalar_kw={'cmap': 'gray'})

In [ ]:
holo_image = mag2exp.x_ray.holography(system.m, fwhm=(15e-9, 15e-9))

In [ ]:
holo_image.mpl.scalar(cmap='RdBu', interpolation='spline16', symmetric_clim=True)

In [ ]:
mag2exp.quick_plots.ltem_defocus(system.m, voltage=300e3, df_length=0.2e-3)

In [ ]:
system.m.norm.k3d.nonzero()

In [ ]:
field_rotator = df.FieldRotator(system.m)
field_rotator.rotate("from_euler", seq="y", angles=10, degrees=True)
field_rotator.field.norm.k3d.nonzero()

In [ ]:
mag2exp.quick_plots.ltem_defocus(field_rotator.field, voltage=300e3, df_length=0.2e-3)